# ДЗ 2

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1MB 7.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419497 sha256=e57d3588d5995151596c82af7382e671eb4431f54365222a2b2ed1a7d4cdc374
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import random
from typing import List

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Recomendation/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] <  data['week_no'].max() - test_size_weeks]
data_test  = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? Зачем он нужен. Используя этот товар мы смещаяем качество рекомендаций. В какую сторону?  Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре.

In [6]:
def precision_at_k_l(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
#     print(f"bought_list: {bought_list}")
#     print(f"recommended_list: {recommended_list}")

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
#     print(flags)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [7]:
def adapt_par(inc: List) -> List:
    return list(map(int, inc.replace("\n", "").replace(",", "").replace("[", "").replace("]", "").split()))

In [8]:
predictions_basic = pd.read_csv('/content/drive/MyDrive/Recomendation/predictions_basic.csv')
predictions_basic.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[932106, 6633318, 962703, 9653988, 8204464]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[829080, 93700, 937166, 1136048, 2333833]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [9]:
col_list = predictions_basic.columns[1:]
col_list

Index(['actual', 'random_recommendation', 'popular_recommendation', 'itemitem',
       'cosine', 'tfidf', 'own_purchases'],
      dtype='object')

In [10]:
for col_n in col_list:
    print(type(predictions_basic[col_n][0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [11]:
for col_n in col_list:
    predictions_basic[col_n] = list(map(adapt_par, predictions_basic[col_n]))
    print(type(predictions_basic[col_n][0][0]))
    
predictions_basic.head(2)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[932106, 6633318, 962703, 9653988, 8204464]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[829080, 93700, 937166, 1136048, 2333833]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [12]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [13]:
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [14]:
user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,26738,26889,26941,27021,27030,27152,27158,27159,27323,27334,27346,27404,27479,27491,27503,27509,27510,27522,27558,27633,27657,27658,27686,27695,27697,27732,27735,27745,27754,27760,...,17249003,17249285,17249614,17249651,17282500,17282612,17283027,17283348,17283671,17283763,17284209,17284296,17284297,17284346,17284401,17284423,17284629,17285065,17291184,17291554,17291665,17319084,17320591,17320698,17320734,17320911,17321083,17321092,17328261,17328345,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
%%time

model = ItemItemRecommender(K=5, num_threads=2) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                       user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                       N=5, # кол-во рекомендаций 
                       filter_already_liked_items=False, 
                       filter_items=None, 
                       recalculate_user=True)


CPU times: user 22.7 s, sys: 113 ms, total: 22.8 s
Wall time: 18.6 s


In [17]:
%%time

predictions_basic['itemitem_noadd'] = predictions_basic['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 123 ms, sys: 934 µs, total: 124 ms
Wall time: 124 ms


In [18]:
f"{sum(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic.itemitem, predictions_basic.actual))):.4f}"

'279.6000'

In [19]:
f"{sum(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic.itemitem_noadd, predictions_basic.actual))):.4f}"

'314.6000'

Дополнительная метка события в списке товаров, конечно, негативно влияет на метрики, т.к. метка события вытесняет релевантные товары.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [20]:
items_weights = data.groupby('item_id')['sales_value'].sum()
items_weights

item_id
25671       20.94
26081        0.99
26093        1.59
26190        1.54
26355        1.98
            ...  
17991689     2.49
17991691     2.49
18000012    19.96
18024155     3.99
18024556    30.51
Name: sales_value, Length: 89051, dtype: float64

In [21]:
min_val = min(items_weights.values[items_weights.values>0])
min_val

8.881784e-16

In [23]:
items_weights.values[items_weights.values==0] = min_val

In [ ]:
items_weights = np.log(items_weights)
items_weights

item_id
25671       3.041661
26081      -0.010050
26093       0.463734
26190       0.431782
26355       0.683097
              ...   
17991689    0.912283
17991691    0.912283
18000012    2.993730
18024155    1.383791
18024556    3.418054
Name: sales_value, Length: 89051, dtype: float64

In [24]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    recs = random.choices(items_weights.index, items_weights.values, k=n)
    
    return random.choices(items_weights.index, items_weights.values, k=n) # recs.tolist()

Сделайте предсказания

In [25]:
%%time

predictions_basic['weighted_rnd'] = \
        predictions_basic.user_id.apply(lambda _: weighted_random_recommendation(items_weights, 5))

CPU times: user 1min 3s, sys: 132 ms, total: 1min 3s
Wall time: 1min 3s


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [26]:
col_list = predictions_basic.columns[1:]
col_list

Index(['actual', 'random_recommendation', 'popular_recommendation', 'itemitem',
       'cosine', 'tfidf', 'own_purchases', 'itemitem_noadd', 'weighted_rnd'],
      dtype='object')

In [27]:
predictions_basic.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_noadd,weighted_rnd
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[932106, 6633318, 962703, 9653988, 8204464]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1127831, 995242, 840361]","[995478, 1027569, 6534178, 1004475, 987311]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[829080, 93700, 937166, 1136048, 2333833]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 826249, 995242]","[963727, 1041476, 6534178, 991886, 910032]"


In [28]:
prec_5 = {}

for col_n in col_list[1:]:
#     print(f"Precision@5 {col_n}: "
#           f"{np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual))):.4f}")
    prec_5[col_n] = np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual)))

for k, v in dict(sorted(prec_5.items(), key=lambda item: item[1])).items():
    print(f"Precision@5: {v:0.4f} - {k}")

Precision@5: 0.0008 - random_recommendation
Precision@5: 0.0244 - weighted_rnd
Precision@5: 0.1329 - cosine
Precision@5: 0.1369 - itemitem
Precision@5: 0.1390 - tfidf
Precision@5: 0.1541 - itemitem_noadd
Precision@5: 0.1552 - popular_recommendation
Precision@5: 0.1797 - own_purchases


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [29]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

#### Random recommendation

In [30]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [31]:
%%time

predictions_basic['random_recommendation_t5k'] = predictions_basic['user_id'].apply(lambda _: random_recommendation(top_5000, n=5))
predictions_basic.head(2)

CPU times: user 974 ms, sys: 962 µs, total: 975 ms
Wall time: 984 ms


#### Weighted random recommender

In [32]:
items_weights.sort_values(ascending=False, inplace=True)
items_weights[:5000]

item_id
6534178    467993.62
6533889     42645.75
1029743     37981.91
6534166     31298.96
1082185     27291.02
             ...    
7410342       279.13
914188        279.13
1036297       279.07
1071845       279.05
973042        279.00
Name: sales_value, Length: 5000, dtype: float64

In [33]:
%%time

predictions_basic['weighted_rnd_t5k'] = \
        predictions_basic.user_id.apply(lambda _: weighted_random_recommendation(items_weights[:5000], 5))

CPU times: user 3.56 s, sys: 2.06 ms, total: 3.56 s
Wall time: 3.57 s


In [34]:
col_list = predictions_basic.columns[1:]
col_list

Index(['actual', 'random_recommendation', 'popular_recommendation', 'itemitem',
       'cosine', 'tfidf', 'own_purchases', 'itemitem_noadd', 'weighted_rnd',
       'random_recommendation_t5k', 'weighted_rnd_t5k'],
      dtype='object')

In [35]:
prec_5 = {}

for col_n in col_list[1:]:
#     print(f"Precision@5 {col_n}: "
#           f"{np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual))):.4f}")
    prec_5[col_n] = np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual)))

for k, v in dict(sorted(prec_5.items(), key=lambda item: item[1])).items():
    print(f"Precision@5: {v:0.4f} - {k}")

Precision@5: 0.0008 - random_recommendation
Precision@5: 0.0061 - random_recommendation_t5k
Precision@5: 0.0244 - weighted_rnd
Precision@5: 0.0355 - weighted_rnd_t5k
Precision@5: 0.1329 - cosine
Precision@5: 0.1369 - itemitem
Precision@5: 0.1390 - tfidf
Precision@5: 0.1541 - itemitem_noadd
Precision@5: 0.1552 - popular_recommendation
Precision@5: 0.1797 - own_purchases


#### ItemItemRecommender

In [36]:
%%time

for i in range(1, 7):

    model = ItemItemRecommender(K=i, num_threads=2) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                           user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                           N=5, # кол-во рекомендаций 
                           filter_already_liked_items=False, 
                           filter_items=None, 
                           recalculate_user=True)

    col_name_str = 'itemitem_noadd_k' + str(i)
    predictions_basic[col_name_str] = predictions_basic['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])


CPU times: user 2min 16s, sys: 520 ms, total: 2min 16s
Wall time: 1min 51s


In [37]:
col_list = predictions_basic.columns[1:]
col_list

Index(['actual', 'random_recommendation', 'popular_recommendation', 'itemitem',
       'cosine', 'tfidf', 'own_purchases', 'itemitem_noadd', 'weighted_rnd',
       'random_recommendation_t5k', 'weighted_rnd_t5k', 'itemitem_noadd_k1',
       'itemitem_noadd_k2', 'itemitem_noadd_k3', 'itemitem_noadd_k4',
       'itemitem_noadd_k5', 'itemitem_noadd_k6'],
      dtype='object')

In [38]:
prec_5 = {}

for col_n in col_list[1:]:
#     print(f"Precision@5 {col_n}: "
#           f"{np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual))):.4f}")
    prec_5[col_n] = np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 5), predictions_basic[col_n], predictions_basic.actual)))

for k, v in dict(sorted(prec_5.items(), key=lambda item: item[1])).items():
    print(f"Precision@5: {v:0.4f} - {k}")

Precision@5: 0.0008 - random_recommendation
Precision@5: 0.0061 - random_recommendation_t5k
Precision@5: 0.0244 - weighted_rnd
Precision@5: 0.0355 - weighted_rnd_t5k
Precision@5: 0.1329 - cosine
Precision@5: 0.1369 - itemitem
Precision@5: 0.1390 - tfidf
Precision@5: 0.1510 - itemitem_noadd_k4
Precision@5: 0.1541 - itemitem_noadd
Precision@5: 0.1541 - itemitem_noadd_k5
Precision@5: 0.1552 - popular_recommendation
Precision@5: 0.1591 - itemitem_noadd_k6
Precision@5: 0.1700 - itemitem_noadd_k3
Precision@5: 0.1797 - own_purchases
Precision@5: 0.2063 - itemitem_noadd_k2
Precision@5: 0.2195 - itemitem_noadd_k1


#### Stack

In [39]:
np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 10), 
                 predictions_basic['own_purchases'], 
                 predictions_basic.actual)))

0.17969311132876264

In [40]:
np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 10), 
                 predictions_basic['popular_recommendation'], 
                 predictions_basic.actual)))

0.15523996082272282

In [41]:
np.mean(list(map(lambda x, y: precision_at_k_l(x, y, 10), 
                 predictions_basic['own_purchases'] + predictions_basic['popular_recommendation'], 
                 predictions_basic.actual)))

0.09635686301944872

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

In [42]:
item_user_matrix = pd.pivot_table(data_train, 
                                  index='item_id', columns='user_id', 
                                  values='quantity',
                                  aggfunc='sum', 
                                  fill_value=0)

In [43]:
item_user_matrix.head(3)

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25671,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26093,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
import tqdm

In [45]:
def ii_cossim(data, ind_src, n=5):
    sim_list = []

    for i in tqdm.tqdm(range(item_user_matrix.shape[0])):
        if i != ind_src:
            sm = item_user_matrix.iloc[i]@item_user_matrix.iloc[ind_src]
            if sm:
                sim_list.append(sm/np.sqrt(sum(item_user_matrix.iloc[i]**2)*sum(item_user_matrix.iloc[ind_src]**2)))

    sim_list.sort(reverse=True)
    return sim_list[:5]

In [46]:
%%time

ii_cossim(item_user_matrix, 0, n=5)

100%|██████████| 86865/86865 [00:35<00:00, 2451.14it/s]

CPU times: user 35.3 s, sys: 193 ms, total: 35.5 s
Wall time: 35.4 s


[0.9428090415820635,
 0.9428090415820635,
 0.816496580927726,
 0.769800358919501,
 0.7257747386024231]